In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -q aitextgen

import logging
logging.basicConfig(
        format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO
    )






In [3]:
!pip install aitextgen
!pip install accelerate -U
!pip uninstall -y pytorch-lightning aitextgen
!pip install pytorch-lightning==1.5.10
!pip install aitextgen==0.5.2
!pip install datasets
!pip uninstall transformers -y
!pip install transformers[torch]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 2.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1
Found existing installation: pytorch-lightning 2.2.5
Uninstalling pytorch-lightning-2.2.5:
  Successfully uninstalled pytorch-lightning-2.2.5
Found existing installation: aitextgen 0.6.0
Uninstalling aitextgen-0.6.0:
  Successfully uninstalled aitextgen-0.6.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 4.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 16.6 MB/s eta 0:00:00a 0:00:01
DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming depende

In [4]:
from aitextgen import aitextgen


2024-07-05 09:04:11.249139: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-05 09:04:11.249246: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-05 09:04:11.429633: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# use GPT Neo
ai = aitextgen(model="EleutherAI/gpt-neo-125M", to_gpu=True)

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256
}



generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

loading configuration file generation_config.json from cache at aitextgen/models--EleutherAI--gpt-neo-125M/snapshots/21def0189f5705e2521767faed922f1f15e7d7db/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256
}



...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

loading file vocab.json from cache at aitextgen/models--EleutherAI--gpt-neo-125M/snapshots/21def0189f5705e2521767faed922f1f15e7d7db/vocab.json
loading file merges.txt from cache at aitextgen/models--EleutherAI--gpt-neo-125M/snapshots/21def0189f5705e2521767faed922f1f15e7d7db/merges.txt
loading file tokenizer.json from cache at aitextgen/models--EleutherAI--gpt-neo-125M/snapshots/21def0189f5705e2521767faed922f1f15e7d7db/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at aitextgen/models--EleutherAI--gpt-neo-125M/snapshots/21def0189f5705e2521767faed922f1f15e7d7db/special_tokens_map.json
loading file tokenizer_config.json from cache at aitextgen/models--EleutherAI--gpt-neo-125M/snapshots/21def0189f5705e2521767faed922f1f15e7d7db/tokenizer_config.json


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset
import torch

# Load the dataset
dataset = load_dataset("codeparrot/xlcost-text-to-code", "C++-program-level")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Filter necessary columns
tokenized_dataset = tokenized_dataset.remove_columns(["text", "code"])

# Load the model
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")
model.resize_token_embeddings(len(tokenizer))

# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,  # Adjust batch size if necessary
    num_train_epochs=2,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
)

# Custom Trainer subclass
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs['input_ids']
        outputs = model(**inputs)
        logits = outputs.logits
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Initialize CustomTrainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

# Enable CUDA assertions
import os
os.environ["TORCH_USE_CUDA_DSA"] = "1"

# Start training
trainer.train()

# Save the trained model
model.save_pretrained('./trained_model')
tokenizer.save_pretrained('./trained_model')


Generating train split:   0%|          | 0/9797 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/909 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/492 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,5.175700
1000,12.677100
1500,10.612800
2000,1.220300
2500,0.338900
3000,0.291100
3500,0.272600
4000,0.266100
4500,0.252300
5000,0.234000


Saving model checkpoint to ./results/checkpoint-9798
Configuration saved in ./results/checkpoint-9798/generation_config.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved in ./trained_model/generation_config.json
tokenizer config file saved in ./trained_model/tokenizer_config.json
Special tokens file saved in ./trained_model/special_tokens_map.json


('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.json',
 './trained_model/merges.txt',
 './trained_model/added_tokens.json',
 './trained_model/tokenizer.json')

In [7]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("/kaggle/working/fine-tuned-gpt-neo-125M")
tokenizer.save_pretrained("/kaggle/working/fine-tuned-gpt-neo-125M")


Configuration saved in /kaggle/working/fine-tuned-gpt-neo-125M/generation_config.json
tokenizer config file saved in /kaggle/working/fine-tuned-gpt-neo-125M/tokenizer_config.json
Special tokens file saved in /kaggle/working/fine-tuned-gpt-neo-125M/special_tokens_map.json


('/kaggle/working/fine-tuned-gpt-neo-125M/tokenizer_config.json',
 '/kaggle/working/fine-tuned-gpt-neo-125M/special_tokens_map.json',
 '/kaggle/working/fine-tuned-gpt-neo-125M/vocab.json',
 '/kaggle/working/fine-tuned-gpt-neo-125M/merges.txt',
 '/kaggle/working/fine-tuned-gpt-neo-125M/added_tokens.json',
 '/kaggle/working/fine-tuned-gpt-neo-125M/tokenizer.json')

In [8]:
from transformers import AutoModelForCausalLM

# Replace 'path_to_your_trained_model_directory' with the actual path
model_path = '/kaggle/working/fine-tuned-gpt-neo-125M'
model = AutoModelForCausalLM.from_pretrained(model_path)


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256
}

loading configuration file /kaggle/working/fine-tuned-gpt-neo-125M/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256
}



In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")

# Example input text
input_text = "C++ code to generate random numbers"

# Tokenize input text
inputs = tokenizer(input_text, return_tensors="pt")


loading file vocab.json from cache at /root/.cache/huggingface/hub/models--EleutherAI--gpt-neo-125M/snapshots/21def0189f5705e2521767faed922f1f15e7d7db/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--EleutherAI--gpt-neo-125M/snapshots/21def0189f5705e2521767faed922f1f15e7d7db/merges.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--EleutherAI--gpt-neo-125M/snapshots/21def0189f5705e2521767faed922f1f15e7d7db/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--EleutherAI--gpt-neo-125M/snapshots/21def0189f5705e2521767faed922f1f15e7d7db/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--EleutherAI--gpt-neo-125M/snapshots/21def0189f5705e2521767faed922f1f15e7d7db/tokenizer_config.json


In [14]:


# Set model to evaluation mode
model.eval()

# Generate text
output = model.generate(
    inputs.input_ids,
    attention_mask=inputs.attention_mask,  # Include attention mask
    max_length=1000,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    early_stopping=True,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

# Decode generated output tokens back to text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:")
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:
C++ code to generate random numbers.

A:
#include <iostream>
using namespace std;
int main()
{
   int n = 100;

  cout << "Enter a random number: "; 
}
void random() {
cin >> n; //random number
cout << endl;//random
n = 0;cOut << n <<endl
for (int i = 1; i < n-1; ++i)
 {  //do something
 }
 cout. coutback();
 return 0 ;
};
//main()


